In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
import lightgbm as lgb

In [2]:
from nlp_pipeline import *

In [47]:
oof21 = pd.read_csv("submissions\\oof_train21.csv")
sub21 = pd.read_csv("submissions\\submission21.csv")
oof22 = pd.read_csv("submissions\\oof_train22.csv")
sub22 = pd.read_csv("submissions\\submission22.csv")
ooflstm4 = pd.read_csv("submissions\\lstm_ft_oof_template.csv")
sublstm4 = pd.read_csv("submissions\\lstm4.csv")
oof29 = pd.read_csv("submissions\\oof_train29.csv")
sub29 = pd.read_csv("submissions\\submission29.csv")
oof31 = pd.read_csv("submissions\\oof_train31.csv")
sub31 = pd.read_csv("submissions\\submission31.csv")
oof33 = pd.read_csv("submissions\\oof_train33.csv")
sub33 = pd.read_csv("submissions\\submission33.csv")
oof35 = pd.read_csv("submissions\\oof_train35.csv")
sub35 = pd.read_csv("submissions\\submission35.csv")

In [48]:
train = pd.read_csv('data\\train.csv').fillna(' ')
test = pd.read_csv('data\\test.csv').fillna(' ')

In [49]:
labels = [c for c in oof21.columns][1:8]

In [86]:
pretrained = "data\\crawl-300d-2M.vec"
feature_funcs = [len, asterix_freq, uppercase_freq, line_change_freq, rep_freq, question_freq]
transforms = [tokenize]
logreg = LogisticRegression(C=0.2, class_weight='balanced', solver='newton-cg', max_iter=10)
logreg.name = "Logistic regression stacker"
gbm = lgb.LGBMClassifier(max_depth=3, metric="auc", num_leaves=20, boosting_type="gbdt", learning_rate=0.1, feature_fraction=0.9, bagging_fraction=0.8, bagging_freq=5, reg_lambda=0.2)
gbm.name = "LightGBM stacker"
models = [gbm]

In [87]:
pipe = NlpPipeline(train, test, "comment_text", labels, feature_funcs, transforms, models, word_index=None, pretrained=pretrained)

In [88]:
pipe.engineer_features()

Engineering features


In [89]:
train_meta = np.hstack([np.array(oof21[labels]), np.array(oof22[labels]), np.array(ooflstm4[labels]), np.array(oof29[labels]), np.array(oof31[labels]), np.array(oof33[labels])])
test_meta = np.hstack([np.array(sub21[labels]), np.array(sub22[labels]), np.array(sublstm4[labels]), np.array(sub29[labels]), np.array(sub31[labels]), np.array(sub33[labels])])

In [90]:
pipe.train_features = np.hstack([pipe.train_features, train_meta])
pipe.test_features = np.hstack([pipe.test_features, test_meta])

In [75]:
param_grid = {'learning_rate': [0.05, 0.1, 0.15],
             'reg_lambda': [0.1,0.2,0.5],
             'num_leaves': [20,30,40]}

In [77]:
from sklearn.model_selection import GridSearchCV

In [78]:
model = GridSearchCV(estimator=gbm, param_grid=param_grid, scoring='roc_auc',
                                 verbose=10, n_jobs=-1, iid=True, refit=True, cv=3)

In [79]:
model.fit(pipe.train_features, train["severe_toxic"])

Fitting 3 folds for each of 27 candidates, totalling 81 fits


[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    7.9s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:   20.3s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:   29.5s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   36.6s
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   50.9s
[Parallel(n_jobs=-1)]: Done  45 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done  75 out of  81 | elapsed:  1.7min remaining:    8.0s
[Parallel(n_jobs=-1)]: Done  81 out of  81 | elapsed:  1.7min finished


GridSearchCV(cv=3, error_score='raise',
       estimator=LGBMClassifier(bagging_fraction=0.8, bagging_freq=5, boosting_type='gbdt',
        class_weight=None, colsample_bytree=1.0, feature_fraction=0.9,
        learning_rate=0.1, max_depth=-1, metric='auc',
        min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
        n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
        random_state=None, reg_alpha=0.0, reg_lambda=0.5, silent=True,
        subsample=1.0, subsample_for_bin=200000, subsample_freq=1),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'learning_rate': [0.05, 0.1, 0.15], 'reg_lambda': [0.1, 0.2, 0.5], 'num_leaves': [20, 30, 40]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='roc_auc', verbose=10)

In [84]:
model.best_params_

{'learning_rate': 0.1, 'num_leaves': 20, 'reg_lambda': 0.2}

In [105]:
gbm.n_estimators = 100

In [106]:
pipe.cross_val()

Cross-validating
LGBMClassifier(bagging_fraction=0.8, bagging_freq=5, boosting_type='gbdt',
        class_weight=None, colsample_bytree=1.0, feature_fraction=0.9,
        learning_rate=0.1, max_depth=3, metric='auc', min_child_samples=20,
        min_child_weight=0.001, min_split_gain=0.0, n_estimators=100,
        n_jobs=-1, num_leaves=20, objective=None, random_state=None,
        reg_alpha=0.0, reg_lambda=0.2, silent=True, subsample=1.0,
        subsample_for_bin=200000, subsample_freq=1)
Cross-validating toxic
roc_auc: 0.9860148341168274
Cross-validating severe_toxic
roc_auc: 0.9912245593862791
Cross-validating obscene
roc_auc: 0.9946645608677009
Cross-validating threat
roc_auc: 0.9918406176476227
Cross-validating insult
roc_auc: 0.9883015430767765
Cross-validating identity_hate
roc_auc: 0.9892622085563243


In [107]:
pipe.cv_scores

{'LightGBM stacker': 0.9902180539419217}

In [108]:
pipe.fit_predict()

Fitting and predicting
Fitting submission classifier for toxic
Fitting submission classifier for severe_toxic
Fitting submission classifier for obscene
Fitting submission classifier for threat
Fitting submission classifier for insult
Fitting submission classifier for identity_hate


In [109]:
pipe.create_submission(oof=False)

Creating submissions


In [15]:
from keras.models import Sequential
from keras.layers import Activation

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [17]:
from keras.layers import Dense, Input

In [23]:
model = Sequential()
model.add(Dense(50, input_dim=42, init="uniform", activation="relu"))
model.add(Dense(50, init="uniform", activation="relu"))
model.add(Dense(6, activation="softmax"))

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(50, input_dim=42, activation="relu", kernel_initializer="uniform")`
  if __name__ == '__main__':


In [26]:
model.compile(loss="binary_crossentropy", optimizer="Adam",metrics=["accuracy"])

In [27]:
model.fit(pipe.train_features, train[labels], batch_size=32, epochs=2, validation_split=0.1)

Train on 143613 samples, validate on 15958 samples
Epoch 1/2
143613/143613 [==============================] - 28s 193us/step - loss: 0.2185 - acc: 0.9650 - val_loss: 0.2175 - val_acc: 0.9642
Epoch 2/2
143613/143613 [==============================] - 23s 157us/step - loss: 0.2170 - acc: 0.9649 - val_loss: 0.2171 - val_acc: 0.9649


In [29]:
preds = model.predict(pipe.test_features, batch_size=1024, verbose=1)

153164/153164 [==============================] - 0s 2us/step


In [31]:
preds

array([[2.5389239e-01, 9.8620430e-02, 2.5308859e-01, 2.3221290e-02,
        2.2590561e-01, 1.4527166e-01],
       [1.4058174e-01, 1.8782757e-01, 1.5639730e-01, 1.7710847e-01,
        1.7134452e-01, 1.6674034e-01],
       [1.3537392e-01, 1.9352107e-01, 1.5800655e-01, 1.7449144e-01,
        1.6828273e-01, 1.7032433e-01],
       ...,
       [1.6681387e-01, 1.7096893e-01, 1.6598742e-01, 1.6512366e-01,
        1.6458720e-01, 1.6651896e-01],
       [1.7414729e-01, 1.6858935e-01, 1.6072893e-01, 1.6618620e-01,
        1.6090053e-01, 1.6944765e-01],
       [5.0065041e-01, 3.5435907e-03, 2.9495752e-01, 1.4399772e-04,
        1.9919479e-01, 1.5097217e-03]], dtype=float32)